# <center> Testing torch-serve FuSA </center>

Este notebook apunta a documentar cómo enviar archivos de audio wav a la API de TorchServe FuSA para hacer inferencias.

Contacto: Matthieu Vernier (mvernier@inf.uach.cl), Victor Várgas (victorvargassandoval93@gmail.com)

# 1. Conexión a la base de datos mongo de FuSA

In [1]:
def get_database():
    from pymongo import MongoClient

    # Creo una conexión usando MongoClient.
    client = MongoClient('45.79.170.31', 27017)

    # Retorno la base de datos FuSA
    db = client['fusa']
    return db

# 2. Query para obtener todos los audios

In [2]:
#Guardo todos los audios en una lista
db = get_database()
audios = []
for audio in db.audios.find():
    audios.append(audio)

# 3. Obtengo sólo un audio de la lista para enviarlo a torch-serve

In [3]:
N=0
selected_audio = audios[N]
selected_audio

{'_id': ObjectId('614954d4779edf6bd07a4f16'),
 'name': 'Hola',
 'description': '',
 'format': 'audio/x-m4a',
 'size': 83594.0,
 'duration': 3.181134,
 'recorded_at': 1632195660,
 'uploaded_at': 1632195796,
 'latitude': -39.84023611383635,
 'longitude': -73.21745933858777,
 'data': '...',
 'recording_device': 'smartphone',
 'user': {'category': 'citizen', 'username': 'demo@fusa.cl'},
 'tags': [{'username': 'demo@fusa.cl', 'source_tags': ['habla']}],
 'chunks': None}

# 4. Escribo un archivo audio tomando la data codificada en base64

In [4]:
import base64

def str_bytes_to_audio(base64_string, result_filename):
    base64_bytes = base64_string.encode("utf-8")        
    with open(result_filename, 'wb') as audio_file:
        byte_content = base64.decodebytes(base64_bytes)
        audio_file.write(byte_content)

In [5]:
import os
folder_results = 'data/'
if not os.path.exists(folder_results):
    os.mkdir(folder_results)
audio_format = selected_audio["format"].split('/')[1].split('-')[1]
result_filename = f"{folder_results}demo.{audio_format}"
base64_string = selected_audio["data"].split(',')[1]
str_bytes_to_audio(base64_string, result_filename) #base64str to audiofile

In [6]:
#Si el formato no es wav, realizar conversión
from pydub import AudioSegment
if audio_format != 'wav':
    result_wav_filename = f"{folder_results}demo.wav"
    track = AudioSegment.from_file(result_filename)
    file_handle = track.export(result_wav_filename, format='wav')
    result_filename = result_wav_filename

In [7]:
#Escuchar el archivo
import IPython.display as ipd
ipd.Audio(result_filename)

# 5. Datos para conectarse con torch-serve

In [8]:
INFERENCE_ADDRESS = 'http://45.79.145.155:8080'
MANAGEMENT_ADDRESS = 'http://45.79.145.155:8081'

In [9]:
import requests
import pprint
#Obtiene el estado del servidor
ping = requests.get(INFERENCE_ADDRESS + '/ping')
print(ping.status_code)
ping.json()

200


{'status': 'Healthy'}

In [10]:
#Obtiene el estado del servidor
models = requests.get(MANAGEMENT_ADDRESS + '/models')
print(models.status_code)
models.json()

200


{'models': [{'modelName': 'fusanet', 'modelUrl': 'fusanet.mar'}]}

In [11]:
#Obtiene información acerca del modelo fusanet
fusanet = requests.get(MANAGEMENT_ADDRESS + '/models/fusanet')
print(fusanet.status_code)
fusanet.json()

200


[{'modelName': 'fusanet',
  'modelVersion': '0.1',
  'modelUrl': 'fusanet.mar',
  'runtime': 'python',
  'minWorkers': 1,
  'maxWorkers': 1,
  'batchSize': 3,
  'maxBatchDelay': 5000,
  'loadedAtStartup': True,
  'workers': [{'id': '9000',
    'startTime': '2021-09-15T18:48:04.336Z',
    'status': 'READY',
    'memoryUsage': 173760512,
    'pid': 56,
    'gpu': False,
    'gpuUsage': 'N/A'}]}]

# 6. Inferencia de modelo fusanet en torch-serve

In [12]:
#Obtiene una predicción modelo fusanet
import time
start = time.time()

files = {'data': open(result_filename,'rb')}
predictions = requests.post(
    INFERENCE_ADDRESS + '/predictions/fusanet', files=files)
print(predictions.status_code)
print(f'{time.time() - start} segundos')
predictions.json()

200
5.96952223777771 segundos


{'humans/others': 0.9995612502098083,
 'mechanical/explosives': 0.0003796792880166322,
 'animal/others': 5.652504478348419e-05,
 'mechanical/cutting': 2.495034323146683e-06,
 'environmental/others': 1.69226822777091e-07,
 'animal/bird': 8.287726060984824e-09,
 'alerts/siren': 6.773256622238932e-09,
 'vehicles/airborne': 2.4242541307728516e-09,
 'alerts/bells': 8.46563819045798e-10,
 'animal/dog': 3.474337206821332e-10,
 'vehicles/others': 1.876943878098558e-10,
 'mechanical/others': 2.889088924692973e-12,
 'environmental/rain': 1.3906906767661986e-13,
 'environmental/wind': 1.468471882721395e-14,
 'alerts/horns': 1.190345062246023e-16}